In [ ]:
import argparse
import numpy as np
import os
import cv2
import tqdm
import torch
import pickle
import copy
import glob

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger

import perspective2d.modeling  # noqa
from perspective2d.utils.predictor import VisualizationDemo
from perspective2d.config import get_perspective2d_cfg_defaults
from perspective2d.utils import draw_perspective_fields, draw_from_r_p_f_cx_cy
from perspective2d.utils.panocam import PanoCam
import matplotlib.pyplot as plt

In [ ]:
MODEL_ID = 'paramnet_gsv_rpfpp' # 'paramnet_gsv_rpfpp'
# MODEL_ID = 'paramnet_gsv_rpf'
model_zoo = {
    # trained on GSV dataset, predicts Perspective Fields + camera parameters (roll, pitch, fov, principal point)
    'paramnet_gsv_rpfpp': {
        'weights': ['https://www.dropbox.com/s/ufdadxigewakzlz/paramnet_gsv_rpfpp.pth'],
        'opts': ['MODEL.WEIGHTS', 'PerspectiveFields_models/paramnet_gsv_rpfpp.pth'],
        'config_file': 'PerspectiveFields_models/paramnet_gsv_rpfpp.yaml',
        'param': True,
    },
    # trained on GSV dataset, predicts Perspective Fields + camera parameters (roll, pitch, fov), assuming centered principal point
    'paramnet_gsv_rpf': {
        'weights': ['https://www.dropbox.com/s/g6xwbgnkggapyeu/paramnet_gsv_rpf.pth'],
        'opts': ['MODEL.WEIGHTS', 'PerspectiveFields_models/paramnet_gsv_rpf.pth'],
        'config_file': 'PerspectiveFields_models/paramnet_gsv_rpf.yaml',
        'param': True,
    },
}
assert MODEL_ID in model_zoo.keys()

for html in model_zoo[MODEL_ID]['weights']:
    if not os.path.exists(os.path.join('PerspectiveFields_models', html.split('/')[-1])):
        !wget -P models/ {html}
def setup_cfg(args):
    cfgs = []
    configs = args['config_file'].split('#')
    weights_id = args['opts'].index('MODEL.WEIGHTS') + 1
    weights = args['opts'][weights_id].split('#')
    for i, conf in enumerate(configs):
        if len(conf) != 0:
            tmp_opts = copy.deepcopy(args['opts'])
            tmp_opts[weights_id] = weights[i]
            cfg = get_cfg()
            get_perspective2d_cfg_defaults(cfg)
            cfg.merge_from_file(conf)
            cfg.merge_from_list(tmp_opts)
            cfg.freeze()
            cfgs.append(cfg)
    return cfgs

perspective_cfg_list = setup_cfg(model_zoo[MODEL_ID])
demo_perspective = VisualizationDemo(cfg_list=perspective_cfg_list)

In [ ]:
def calibrate(net, img):
    pred = net.run_on_image(img)
#     print(pred)
    if 'pred_general_vfov' not in pred.keys():
        pred['pred_general_vfov'] = pred['pred_vfov']
    if 'pred_rel_cx' not in pred.keys():
        pred['pred_rel_cx'] = torch.FloatTensor([0])
    if 'pred_rel_cy' not in pred.keys():
        pred['pred_rel_cy'] = torch.FloatTensor([0])

    r_p_f_rad = np.radians(
        [
            pred['pred_roll'].cpu().item(),
            pred['pred_pitch'].cpu().item(),
            pred['pred_general_vfov'].cpu().item(),
        ]
    )
    cx_cy = [
        pred['pred_rel_cx'].cpu().item(),
        pred['pred_rel_cy'].cpu().item(),
    ]
#     print(f"roll {pred['pred_roll'].cpu().item() :.2f}, pitch {pred['pred_pitch'].cpu().item() :.2f}, fov {pred['pred_general_vfov'].cpu().item() :.2f}")
#     print(f"principal point {pred['pred_rel_cx'].cpu().item() :.2f} {pred['pred_rel_cy'].cpu().item() :.2f} ")
    return r_p_f_rad, cx_cy


In [ ]:
video_id_list = ['001', '003', '005', '006', '007', '008', '009', '011', '012', '013', '014', '015', '016', '017', '019', '020', '023', '025', '027', '034', '036', '039', '040', '043', '044', '046', '048', '049', '050', '051', '053', '054', '055', '056', '058', '059', '060', '066', '067', '068', '069', '070', '071', '073', '074', '075', '076', '077', '080', '085', '086', '087', '088', '090', '091', '092', '093', '094', '095', '098', '099', '105', '108', '110', '112', '114', '115', '116', '117', '118', '125', '127', '128', '129', '130', '131', '132', '135', '136', '141', '146', '148', '149', '150', '152', '154', '156', '158', '159', '160', '161', '164', '167', '169', '170', '171', '172', '175', '178', '179']
calibrates = {}
image_1st, r_p_f_rad_mean, center_mean = {}, {}, {}

for video_id in video_id_list:
    calibrates[video_id] = {}
    images = sorted(glob.glob(os.path.join('/mnt/e/Datasets/Intersections/images/annotated/%s/unmasked' % video_id, '*.jpg')))
    print('%s: %d images' % (video_id, len(images)))
    images = [{'file_name': x, 'img':  read_image(x, format="BGR")} for x in images]
    calibrates[video_id]['image_1st'] = images[0]['img']
    for img in images:
        r_p_f_rad, cx_cy = calibrate(demo_perspective, img['img'])
        img['r_p_f_rad'], img['cx_cy'] = r_p_f_rad, cx_cy
    calibrates[video_id]['r_p_f_rad'], calibrates[video_id]['center'] = [], []
    for img in images:
        calibrates[video_id]['r_p_f_rad'].append(r_p_f_rad)
        calibrates[video_id]['center'].append(cx_cy)
    calibrates[video_id]['r_p_f_rad'] = np.stack(calibrates[video_id]['r_p_f_rad'], axis=0).mean(axis=0)
    calibrates[video_id]['center'] = np.stack(calibrates[video_id]['center'], axis=0).mean(axis=0)
    print(calibrates[video_id]['r_p_f_rad'], calibrates[video_id]['center'])

In [ ]:
def get_horizon_ratio(h, fov_h, pitch):
    assert pitch - fov_h / 2 > -np.pi / 2
    if pitch < 0:
        if abs(pitch) > fov_h / 2:
            return -1, np.cos(np.pi / 2 - abs(pitch) - fov_h / 2) / np.cos(np.pi / 2 - abs(pitch) + fov_h / 2)
        else:
            return int(h / 2 + np.tan(pitch) / np.tan(fov_h / 2) * h / 2), -1
    else: # camera not likely points up, fall back to pitch=0
        return int(h / 2), -1

for video_id in video_id_list:
    _, pitch, fov_diag = calibrates[video_id]['r_p_f_rad']
    h, w, _ = calibrates[video_id]['image_1st'].shape
    diag = (w ** 2 + h ** 2) ** 0.5
    fov_w, fov_h = np.arctan(np.tan(fov_diag / 2) * w / diag) * 2, np.arctan(np.tan(fov_diag / 2) * h / diag) * 2
    calibrates[video_id]['horizon'], calibrates[video_id]['ratio'] = get_horizon_ratio(h, fov_h, pitch)

In [ ]:
max_ratio, min_ratio = 3, 2

for video_id in ['001', '019', '091', '016', '175', '036']:
    horizon = calibrates[video_id]['horizon']
    if horizon > 0:
        im_upper, im_lower = calibrates[video_id]['image_1st'][:horizon, :, :], calibrates[video_id]['image_1st'][horizon:, :, :]
        im_upper_unwarped = cv2.resize(im_upper, (int(im_upper.shape[1] * max_ratio), int(im_upper.shape[0] * max_ratio)), cv2.INTER_LINEAR)
        h_lower, w_lower, _ = im_lower.shape
        pt_corners = np.float32([[0, 0], [w_lower, 0], [w_lower, h_lower], [0, h_lower]])
        pt_unwarp = np.float32([[0, 0], \
                                [w_lower * max_ratio, 0], \
                                [w_lower * (max_ratio + 1) / 2, h_lower * max_ratio / 2], \
                                [w_lower * (max_ratio - 1) / 2, h_lower * max_ratio / 2]])
        M = cv2.getPerspectiveTransform(pt_corners, pt_unwarp)
        im_lower_unwarped = cv2.warpPerspective(im_lower, M, \
                                                (int(w_lower * max_ratio), int(h_lower * max_ratio / 2)))
        unwarped = np.concatenate([im_upper_unwarped, im_lower_unwarped], axis=0)
        plt.figure(figsize=(12,12))
        plt.imshow(unwarped); plt.axis('off'); plt.tight_layout()
        plt.show()
    else:
        h, w, _ = calibrates[video_id]['image_1st'].shape
        ratio = max(min_ratio, min(max_ratio, calibrates[video_id]['ratio']))
        pt_corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
        pt_unwarp = np.float32([[0, 0], [w * ratio, 0], [w * (ratio + 1) / 2, h * ratio / 2], [w * (ratio - 1) / 2, h * ratio / 2]])
        M = cv2.getPerspectiveTransform(pt_corners, pt_unwarp)
        unwarped = cv2.warpPerspective(calibrates[video_id]['image_1st'], M, (int(w * ratio), int(h * ratio / 2)))
        plt.figure(figsize=(12,12))
        plt.imshow(unwarped); plt.axis('off'); plt.tight_layout()
        plt.show()